In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import IPython
import pandas as pd

import os
os.chdir("F:\M2\Time Series Classification Project")

## Importing data

In [3]:
dataloc  = "Data/genres_original/"

genres = os.listdir(dataloc)

musiclist = []
genrelist = []
srlist = []

musictrain = []
genretrain = []
srtrain = []
musictest = []
genretest = []
srtest = []

print("Reading songs")
for genre in genres:
    for i, music in enumerate(os.listdir(dataloc + genre)):
        try:
            music_time_series, sr = librosa.load(dataloc + genre + "/" + music)
        except Exception as e:
            print(f"Error loading {music}: {e}")
            continue
        srlist.append(sr)
        musiclist.append(music_time_series)
        genrelist.append(genre)

        if i<25: # 25-75 test train split
            musictest.append(music_time_series)
            genretest.append(genre)
            srtest.append(sr)
        else:
            musictrain.append(music_time_series)
            genretrain.append(genre)
            srtrain.append(sr)

print("Creating dataframe")
fulldata = pd.DataFrame({'Music': musiclist, 'Genre': genrelist, "SampleRate": srlist})

print("Saving dataframe")
fulldata.to_csv("Data/fulldata.csv", index=False)

print("Saving train and test data")
train = pd.DataFrame({"Music" : musictrain, "Genre" : genretrain, "SampleRate": srtrain})
test = pd.DataFrame({"Music" : musictest, "Genre" : genretest, "SampleRate": srtest})
train.to_csv("Data/train.csv", index=False)
test.to_csv("Data/test.csv", index=False)

Reading songs


C:\Users\melvi\AppData\Local\Temp\ipykernel_19768\3135833796.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  music_time_series, sr = librosa.load(dataloc + genre + "/" + music)
C:\Users\melvi\AppData\Roaming\Python\Python39\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error loading jazz.00054.wav: 
Creating dataframe
Saving dataframe
Saving train and test data


## SAX Implementation

In [107]:
letters = np.array(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"])

def SAX(time_series, w, alpha):
    time_series = np.array(time_series)
    time_series = (time_series - np.mean(time_series)) / np.std(time_series)

    # PAA
    PAA=[]
    for word_index in range(w):
        ceiling = min(len(time_series), int((word_index+1)*len(time_series)/w))
        PAA.append(np.mean(time_series[int(word_index*len(time_series)/w):ceiling]))
    
    lookup = np.quantile(time_series, np.arange(1,alpha)*(1/alpha) )
    
    sax = np.digitize(PAA, lookup)

    return list(sax)

def SAX_window(time_series, w, alpha, l):
    sax_sentence = []
    previous_word = np.full(w,-1)

    for i in range(0, len(time_series), l):
        ceiling = min(len(time_series), (i+1)*l )
        sax_word = SAX(time_series[i : ceiling], w, alpha)

        npsaxword = np.array(sax_word)
        if np.sum(npsaxword - previous_word) == 0: # Remove repeated words for reasons
            continue
        
        if i > 0:
            sax_sentence.append(" ")
        sax_sentence.extend(sax_word)
        previous_word = sax_word

    phrase = ""
    for letter in sax_sentence:
        if isinstance(letter, np.int64):
            phrase += letters[letter]
        else :
            phrase += letter
    return phrase

In [109]:
nptrainmusic = train["Music"].to_numpy()

testmusic = nptrainmusic[14]

# Pre-process eliminating noise
#testmusic = librosa.effects.preemphasis(testmusic)

sentence = SAX_window(testmusic, w = 16, alpha = 4, l = int(.2*len(testmusic)+1 ))
print(sentence)

bbbbcbbbbcbbbcbb_bbbbbbbbbbbbbbbb


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

# Function to convert time series to SAX representation
def time_series_to_sax(time_series, w, alpha):
    return ' '.join(map(str, SAX_window(time_series, w, alpha, int(.2*len(time_series)+1))))

# Convert the training and test data to SAX representation
train_sax = [time_series_to_sax(ts, w=16, alpha=4) for ts in train['Music']]
test_sax = [time_series_to_sax(ts, w=16, alpha=4) for ts in test['Music']]

# Filter out empty SAX representations and corresponding labels
train_sax, train_labels = zip(*[(sax, label) for sax, label in zip(train_sax, train['Genre']) if sax.strip()])
test_sax, test_labels = zip(*[(sax, label) for sax, label in zip(test_sax, test['Genre']) if sax.strip()])

# Create a pipeline with CountVectorizer and LogisticRegression
seql_pipeline = make_pipeline(
    CountVectorizer(),
    LogisticRegression(max_iter=1000)
)

# Train the model
print("Training the model")
seql_pipeline.fit(train_sax, train_labels)

# Predict on the test set
test_predictions = seql_pipeline.predict(test_sax)

# Evaluate the model
accuracy = (test_predictions == test_labels).mean()
print(f"Test Accuracy: {accuracy:.2f}")